In [5]:
'''
Column names in the CSV file:
1. id - string
2. name - string
3. popularity - int
4. duration_ms - int
5. explicit - int (boolean)
6. artists - array of strings
7. id_artists - array of strings
8. release_date - date or string? Can be a date or just a year value
9. danceability - double
10. energy - double
11. key - int
12. loudness - double
13. mode - int (boolean)
14. speechiness - double
15. acousticness - double
16. instrumentalness - double
17. liveness - double
18. valence - double
19. tempo - double
20. time_signature - int
'''

'\nColumn names in the CSV file:\n1. id - string\n2. name - string\n3. popularity - int\n4. duration_ms - int\n5. explicit - int (boolean)\n6. artists - array of strings\n7. id_artists - array of strings\n8. release_date - date or string? Can be a date or just a year value\n9. danceability - double\n10. energy - double\n11. key - int\n12. loudness - double\n13. mode - int (boolean)\n14. speechiness - double\n15. acousticness - double\n16. instrumentalness - double\n17. liveness - double\n18. valence - double\n19. tempo - double\n20. time_signature - int\n'

In [ ]:
#import psycopg2
import pandas as pd

# conn = psycopg2.connect("host=ucsd-postgresql-sfo2-do-user-18847016-0.m.db.ondigitalocean.com port=25060 dbname=spotify_dataset user=doadmin password=AVNS_kLeTuGCSss8LYr4Ocrp sslmode=require") 
# cursor = conn.cursor()

# Before, I used the Kaggle API to download the dataset. Now, I will use the CSV file that I downloaded from Kaggle. 
path = "C:kaggle_api_saved_path\\tracks.csv"
df = pd.read_csv(path)
print(df.dtypes)


id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object


In [3]:
import ast

df['id'] = df['id'].astype(str)
df['name'] = df['name'].astype(str)
df['popularity'] = df['popularity'].astype(int)
df['duration_ms'] = df['duration_ms'].astype(int)
df['explicit'] = df['explicit'].astype(bool)  # Convert 0/1 to boolean
df['artists'] = df['artists'].apply(ast.literal_eval).apply(lambda x: list(map(str, x)) if isinstance(x, list) else [])
df['id_artists'] = df['id_artists'].apply(ast.literal_eval).apply(lambda x: list(map(str, x)) if isinstance(x, list) else [])
df['release_date'] = df['release_date'].astype(str)  # Leave as TEXT for flexibility
df['danceability'] = df['danceability'].astype(float)
df['energy'] = df['energy'].astype(float)
df['key'] = df['key'].astype(int)
df['loudness'] = df['loudness'].astype(float)
df['mode'] = df['mode'].astype(bool)  # Convert 0/1 to boolean
df['speechiness'] = df['speechiness'].astype(float)
df['acousticness'] = df['acousticness'].astype(float)
df['instrumentalness'] = df['instrumentalness'].astype(float)
df['liveness'] = df['liveness'].astype(float)
df['valence'] = df['valence'].astype(float)
df['tempo'] = df['tempo'].astype(float)
df['time_signature'] = df['time_signature'].astype(int)

In [4]:
def fix_array_format(df, column_name):
    """
    Converts a column with array-like strings to PostgreSQL-compliant array literals.

    Args:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The name of the column to fix.

    Returns:
        pd.DataFrame: The modified DataFrame.
    """
    def to_postgres_array(value):
        if value in (None, '', pd.NA):  # Handle null or empty values explicitly
            return '{}'
        try:
            # Ensure the value is a Python list
            if isinstance(value, str):
                # Parse string to list
                value = ast.literal_eval(value)
            if isinstance(value, list):
                # Convert list to PostgreSQL array literal
                return '{' + ','.join(['"' + str(x).replace('"', '\\"') + '"' for x in value]) + '}'
            else:
                return '{}'
        except (ValueError, SyntaxError):
            return '{}'  # Default to an empty array for malformed data

    # Apply the transformation to the specified column
    df[column_name] = df[column_name].apply(to_postgres_array)
    return df

In [5]:
df = fix_array_format(df, 'artists')
df = fix_array_format(df, 'id_artists')

In [6]:
def fix_encoding_issues(text):
    """
    Fixes misencoded text by detecting and re-decoding UTF-8 bytes
    misinterpreted as WIN1252.
    
    Args:
        text (str): The input text.
        
    Returns:
        str: The corrected text.
    """
    if not isinstance(text, str):
        return text  # Ignore non-string values
    try:
        # Encode the text as WIN1252 bytes, then decode as UTF-8
        return text.encode('windows-1252').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # Return the original text if decoding fails

# Apply to a DataFrame column
df['artists'] = df['artists'].apply(fix_encoding_issues)
df['id_artists'] = df['id_artists'].apply(fix_encoding_issues)

In [9]:
output_path = "C:\\Users\\pcere\\Downloads\\transformed_tracks.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Transformed DataFrame has been saved to: {output_path}")

Transformed DataFrame has been saved to: C:\Users\pcere\Downloads\transformed_tracks.csv
